In [ ]:
##Final

import nest_asyncio
import threading, json, re
from typing import List
from fastapi import FastAPI, Query
from pydantic import BaseModel
import httpx
import g4f

nest_asyncio.apply()

TMDB_API_KEY = "YOUR_API_KEY"
TMDB_BASE_URL = "https://api.themoviedb.org/3"


# Response Model

class MovieResponse(BaseModel):
    title: str
    rating: float
    overview: str
    release_year: int | None
    genres: List[str]



app = FastAPI()

#  Extract title from query using g4f

async def get_title_from_query(query: str) -> str | None:
    prompt = f"""
    Extract the exact movie title from this user query. Return ONLY the title as a JSON:
    {{
      "title": "Movie Title Here"
    }}
    Query: "{query}"
    """
    for _ in range(3):  # retry 3 times
        try:
            response = g4f.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role":"user","content":prompt}]
            )
            # Extract JSON safely
            match = re.search(r'\{.*\}', response, re.DOTALL)
            if not match:
                continue
            data = json.loads(match.group(0))
            title = data.get("title")
            if title:
                return title.strip()
        except Exception as e:
            print("LLM parse error:", e)
            continue
    return None


# TMDb search

async def search_tmdb_by_title(title: str) -> dict:
    async with httpx.AsyncClient() as client:
        # Search movie
        r = await client.get(f"{TMDB_BASE_URL}/search/movie", params={"api_key": TMDB_API_KEY, "query": title})
        results = r.json().get("results", [])
        if not results:
            return {"title": "Not found", "rating": 0.0, "overview": "", "release_year": None, "genres": []}
        
        movie = results[0]
        
        # Get genres mapping
        genre_resp = await client.get(f"{TMDB_BASE_URL}/genre/movie/list", params={"api_key": TMDB_API_KEY})
        genre_list = genre_resp.json().get("genres", [])
        genre_mapping = {g["id"]: g["name"] for g in genre_list}
        genres = [genre_mapping.get(gid) for gid in movie.get("genre_ids", [])]

        return {
            "title": movie.get("title"),
            "rating": movie.get("vote_average", 0.0),
            "overview": movie.get("overview", ""),
            "release_year": int(movie.get("release_date", "0000-00-00")[:4]) if movie.get("release_date") else None,
            "genres": genres
        }


@app.get("/movie", response_model=MovieResponse)
async def get_movie(query: str = Query(..., description="Natural language movie query")):
    print("Query:", query)
    title = await get_title_from_query(query)
    if not title:
        return {"title": "Not found", "rating": 0.0, "overview": "", "release_year": None, "genres": []}
    return await search_tmdb_by_title(title)


def run_server(port=8002):
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=port, log_level="info")

thread = threading.Thread(target=run_server, args=(8002,), daemon=True)
thread.start()


INFO:     Started server process [15784]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8002 (Press CTRL+C to quit)


In [2]:
import time, requests
time.sleep(2)  # wait for server to start

queries = [
    "What was the rating for Avatar?",
    "What was the rating for that Matt Damon poker movie?",
    "What was the rating for Inception?",
    "What was the rating for the poker movie starring Ed Norton and John Malkovich?"
    
]

for q in queries:
    res = requests.get("http://127.0.0.1:8002/movie", params={"query": q})
    print(f"Query: {q}")
    print(res.json(), "\n")


Query: What was the rating for Avatar?
g4f is up-to-date (version 0.6.3.4).
INFO:     127.0.0.1:51268 - "GET /movie?query=What+was+the+rating+for+Avatar%3F HTTP/1.1" 200 OK
Query: What was the rating for Avatar?
{'title': 'Avatar', 'rating': 7.594, 'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.', 'release_year': 2009, 'genres': ['Action', 'Adventure', 'Fantasy', 'Science Fiction']} 

Query: What was the rating for that Matt Damon poker movie?
INFO:     127.0.0.1:51273 - "GET /movie?query=What+was+the+rating+for+that+Matt+Damon+poker+movie%3F HTTP/1.1" 200 OK
Query: What was the rating for that Matt Damon poker movie?
{'title': 'Rounders', 'rating': 7.008, 'overview': 'A young reformed gambler must return to playing big stakes poker to help a friend pay off loan sharks.', 'release_year': 1998, 'genres': ['Drama', 'Crime']} 

Query: What was the ra